In [1]:
import os
import pandas as pd
os.chdir('/home/aplstudent/Max_Dickinson')


In [2]:
#the goal right now is to get data with timeseries and categories and target to be usable
##then i will clean it by dropping all the rows that have a nan in them
#then i will clean it by dropping all the columns that have a nan in them

In [3]:
def loaddata():
        data = pd.read_csv('train_users_2.csv')
        datatest  = pd.read_csv('test_users.csv')
        return [data , datatest]
data , datatest = loaddata()

In [4]:
## get years and months as a number so it can be a category 

def dataAugment(data , train = False): 
    def getyear(x):
        if type(x) == float:
            return -1
        else :
            return int(x[0:4])
    def getmonth(x):
        if type(x) == float:
            return -1
        else :
            return int(x[5:7])
    def getday(x):
        if type(x) == float:
            return -1
        else :
            return int(x[8:11])
    def getorder(x):
        return( (x[0:4],x[5:7],x[8:11]) )
    def getdate(x):
        return datetime.date(x).toordinal()
    def dayssince(x):
        import matplotlib.dates as dates
        return dates.datestr2num(x)
    data['year_created'] = data['date_account_created'].apply(getyear)
    data['month_created'] = data['date_account_created'].apply(getmonth)
    data['date_created'] = data['date_account_created'].apply(getday)
    data['num_created'] = data['date_account_created'].apply(dayssince)
    if train ==True:
        data['year_booked'] = data['date_first_booking'].apply(getyear)
        data['month_booked'] = data['date_first_booking'].apply(getmonth)
        data['day_booked'] = data['date_first_booking'].apply(getday)
        data['num_booked'] = data['date_first_booking'].dropna().apply(dayssince) #subset of data
    return data
out= dataAugment(data , train =True)
outtest = dataAugment(datatest)

In [ ]:
import numpy as np
def createohe(data):
    #get common axis names
    print("runing")
    ohe = pd.get_dummies(data[['date_created', 'month_created', 'month_booked' , 'day_booked' , 'gender' , 'signup_method', 'signup_flow' , 'language' , 'affiliate_channel' , 'affiliate_provider' , 'first_affiliate_tracked', 'signup_app', 'first_device_type' , 'first_browser']])
    #ohetest = pd.get_dummies(datatest[['date_created', 'month_created' , 'gender' , 'signup_method', 'signup_flow' , 'language' , 'affiliate_channel' , 'affiliate_provider' , 'first_affiliate_tracked', 'signup_app', 'first_device_type' , 'first_browser']])
    #comb = pd.concat((ohe.T, ohetest.T) , join = 'outer')
    #add age and number time columns
    age = data[data['age'] < 100].merge(data[data['age']>14] , how = 'inner' , on = 'age') #subset of data
    minda = pd.concat((ohe, age.age), axis = 1 , join = 'outer' ) 
    mindat = pd.concat(( minda, data['num_created']), axis = 1) #add sequential dates
    xtrain = pd.concat(( mindat, data['num_booked']), axis = 1 , join = 'outer') #add sequential date booked
    #splitintothree
    #datad = mindata.dropna(axis = 1 , how= 'any') ##drop na's in all columns
    ##age = datatest[datatest['age'] < 100].merge(datatest[datatest['age']>14] , how = 'inner' , on = 'age') #subset of data
    #minda = pd.concat((comb.T, age.age), axis = 1 , join = 'outer' ) 
    #mindat = pd.concat(( minda, datatest['num_created']), axis = 1) #add sequential dates
    ##print("runing")
    #xtest = pd.concat(( mindat, datatest['num_booked']), axis = 1 , join = 'outer')
    #datadr = mindata.dropna(axis = 0 , how= 'any') ##drop na's in rows
    xtrain = xtrain.fillna(-1)
    #xtest = xtest.fillna(-1)
    return xtrain

xtrain = createohe(out)  
xtest = createohe(outtest)

In [ ]:
def CountryCleaner(x):
    if x == 'NDF':
        x = 12
    elif x == 'US':
        x=0
    elif x == 'FR':
        x=1
    elif x == 'CA':
        x=2
    elif x == 'GB':
        x=3
    elif x == 'ES':
        x=4
    elif x == 'IT':
        x=5
    elif x == 'PT':
        x=7
    elif x == 'NL':
        x=8
    elif x =='AU':
        x=11
    elif x =='NDF':
        x=10
    elif x =='DE':
        x=9
    elif x =='other':
        x =13
    return x

In [ ]:
def mergedata(datadrop , data):
    allt = data['country_destination'].apply(CountryCleaner)
    dropcol = pd.concat((datadrop,allt) ,axis = 1 , join = 'inner')
    targets = dropcol['country_destination']
    train = dropcol.drop('country_destination' , 1)
    return train, targets
#col ,coltarg = mergedata(datac, out) 
#row, rowtarg = mergedata(datar, out)
xtrain , ytrain = mergedata(xtrain, out)

In [12]:
data.shape

(213451, 24)

In [5]:
#get all ages between 14 and 100
ohe = pd.get_dummies(out[['date_created', 'month_created', 'month_booked' , 'day_booked' , 'gender' , 'signup_method', 'signup_flow' , 'language' , 'affiliate_channel' , 'affiliate_provider' , 'first_affiliate_tracked', 'signup_app', 'first_device_type' , 'first_browser']])
ohetest = pd.get_dummies(outtest[['date_created', 'month_created' , 'gender' , 'signup_method', 'signup_flow' , 'language' , 'affiliate_channel' , 'affiliate_provider' , 'first_affiliate_tracked', 'signup_app', 'first_device_type' , 'first_browser']])
#comb = pd.concat((ohe.T, ohetest.T) , join = 'outer')
print("runing")

runing


In [6]:
#age = out[out['age'] < 100].merge(out[out['age']>14] , how = 'inner' , on = 'age') #subset of data
age = pd.concat((out[out['age'] < 100] , out[out['age']>14]) , join = 'inner', axis =1) #subset of data


In [7]:

minda = pd.concat((comb.T, age.age), axis = 1 , join = 'outer' ) 


NameError: name 'comb' is not defined

In [8]:
mindat = pd.concat(( minda, out['num_created']), axis = 1) #add sequential dates
xtrain = pd.concat(( mindat, out['num_booked']), axis = 1 , join = 'outer') #add sequential date booked

NameError: name 'minda' is not defined

In [9]:

#splitintothree
print("runing")
#datad = mindata.dropna(axis = 1 , how= 'any') ##drop na's in all columns
#age = outtest[outtest['age'] < 100].merge(outtest[outtest['age']>14] , how = 'inner' , on = 'age') #subset of data
minda = pd.concat((comb.T, age.age), axis = 1 , join = 'outer' ) 
xtest = pd.concat(( minda, outtest['num_created']), axis = 1) #add sequential datesprint("runing")
#xtest = pd.concat(( mindat, datatest['num_booked']), axis = 1 , join = 'outer')
#datadr = mindata.dropna(axis = 0 , how= 'any') ##drop na's in rows
print("runing")
xtrain = xtrain.fillna(-1)
xtest = xtest.fillna(-1)

runing


NameError: name 'comb' is not defined

In [ ]:
def gausianfit(data,targets):
    ##i want to make a learning curve
    from sklearn.naive_bayes import GaussianNB
    from sklearn.pipeline import Pipeline, FeatureUnion
    from sklearn.feature_selection import SelectKBest
    from sklearn.pipeline import Pipeline
    from sklearn.svm import SVC
    from sklearn.decomposition import PCA
    from sklearn.naive_bayes import GaussianNB
    from sklearn.grid_search import GridSearchCV
    from sklearn.preprocessing import Normalizer
    #split train and validation sets
    valid = data.loc[0:50000 , :]
    train = data.loc[50000:: , :]
    validy = targets.loc[0:50000 ]
    trainy= targets.loc[50000::]
    combined_features = FeatureUnion([("norm" ,Normalizer()),("pca", PCA()), ("univ_select", SelectKBest())])
    estimators = [ ('comb_features' ,combined_features) , ('gnb', GaussianNB())]
    clf = Pipeline(estimators)
    
    param_grid = dict(comb_features__univ_select__k=[4,10] , comb_features__pca__n_components=[2,8])
    # Maybe some original features where good, too?
    # Build estimator from PCA and Univariate selection:
    grid_search = GridSearchCV(clf, param_grid=param_grid, verbose=10)
    grid_search.fit(train , trainy).predict_proba(xtest) 

        
    id_test = datatest['id']
    ids = []  #list of ids
    cts = []  #list of countries
    for i in range(len(id_test)):
        idx = id_test[i]
        ids += [idx] * 5
        cts += le.inverse_transform(np.argsort(y_pred[i])[::-1])[:5].tolist()

    #Generate submission
    sub = pd.DataFrame(np.column_stack((ids, cts)), columns=['id', 'country'])
    sub.to_csv('Air.csv',index=False)
    return
#gausianfit(col, coltarg)
#gausianfit(row, rowtarg)
gausianfit(xtrain, ytrain)

In [ ]:
def gausianfit(data):
    ##i want to make a learning curve
    from sklearn.naive_bayes import GaussianNB
    from sklearn.pipeline import Pipeline, FeatureUnion
    from sklearn.feature_selection import SelectKBest
    from sklearn.pipeline import Pipeline
    from sklearn.svm import SVC
    from sklearn.decomposition import PCA
    from sklearn.naive_bayes import GaussianNB
    from sklearn.grid_search import GridSearchCV
    from sklearn.preprocessing import Normalizer
    #split train and validation sets
    #valid = data.loc[0:50000 , :]
    #train = data.loc[50000:: , :]
    #validy = targets.loc[0:50000 ]
    #trainy= targets.loc[50000::]
    
    
    combined_features = FeatureUnion([("norm" ,Normalizer()),("pca", PCA()), ("univ_select", SelectKBest())])
    estimators = [ ('comb_features' ,combined_features) , ('gnb', GaussianNB())]
    clf = Pipeline(estimators)
    
    param_grid = dict(comb_features__univ_select__k=[4,10] , comb_features__pca__n_components=[2,8])
    # Maybe some original features where good, too?
    # Build estimator from PCA and Univariate selection:
    grid_search = GridSearchCV(clf, param_grid=param_grid, verbose=10)
    grid_search.predict_proba(data) 

        
    id_test = data['id']
    ids = []  #list of ids
    cts = []  #list of countries
    for i in range(len(id_test)):
        idx = id_test[i]
        ids += [idx]
        cts += le.inverse_transform(np.argsort(y_pred[i])[::-1]).tolist()

    #Generate submission
    sub = pd.DataFrame(np.column_stack((ids, cts)), columns=['id', 'country'])
    sub.to_csv('Air.csv',index=False)
    return
#gausianfit(col, coltarg)
#gausianfit(row, rowtarg)
gausianfit(datatest)

In [ ]:
train.shape

In [ ]:

dropcol = pd.concat((datac,allt) ,axis = 1 , join = 'inner')
droprow = pd.concat((datar , allt) , axis = 1 , join = 'inner')
filldata = pd.concat((filldata , allt) , axis = 1 , join= 'inner')

In [ ]:
targets = dropcol['country_destination']
terg = droprow['country_destination']
filltargets = data['country_destination']


train = dropcol.drop('country_destination' , 1)
train2 = droprow.drop('country_destination', 1)
filltargets = filldata.drop('country_destination', 1)

In [ ]:
clf.fit(train , filltargets).score(valid, validy)

In [ ]:
def throwNan(data):
    import numpy as np
    #dummies of categorical and times
    mind = pd.get_dummies(nndata[['date_created', 'month_created', 'month_booked' , 'day_booked' , 'gender' , 'signup_method', 'signup_flow' , 'language' , 'affiliate_channel' , 'affiliate_provider' , 'first_affiliate_tracked', 'signup_app', 'first_device_type' , 'first_browser']])
    minda = pd.DataFrame(np.empty((0,0)))
    minda = pd.concat(( mind, age), axis = 1 , join = 'inner' )  # this will drop all the na columns
    mindat = pd.concat(( minda, datenums), axis = 1) #add sequential dates
    mindata = pd.concat(( mindat, numbooked), axis = 1 , join = 'inner') #add sequential date booked
    mindata2 = pd.concat((mindata, mn) , join = 'inner' , axis = 1)

In [ ]:
def dayssince(x):
    import matplotlib.dates as dates
    return dates.datestr2num(x)
data['num_created'] = data['date_account_created'].apply(dayssince)
data['num_booked'] = data['date_first_booking'].dropna().apply(dayssince) #subset of data

In [ ]:
droprow.axes

In [ ]:
def onehots():
    ohe = pd.get_dummies(data[['gender' , 'signup_method', 'signup_flow' , 'language' , 'affiliate_channel' , 'affiliate_provider' , 'first_affiliate_tracked', 'signup_app', 'first_device_type' , 'first_browser' ]])
    ohetest = pd.get_dummies(datatest[['gender' , 'signup_method', 'signup_flow' , 'language' , 'affiliate_channel' , 'affiliate_provider' , 'first_affiliate_tracked', 'signup_app', 'first_device_type' , 'first_browser' ]])
    return ohe

In [ ]:
def comonlevels(ohe,ohetest):
        ##now i can train on ohe and test on testdata because they have the same number of columns and the same name of columns
        inboth = []
        for i in range(len(ohetest.axes[1])):
            if ohetest.axes[1][i]  in  ohe.axes[1]:
                inboth.append(ohetest.axes[1][i])
        intrain = ohe[inboth].axes[1]
        ina = ohe[inboth]
        out = []
        for i in range(len(ohe.axes[1])):
            if ohe.axes[1][i] not in intrain:
                out.append(ohe.axes[1][i])
        for i in out:
            ina.loc[:,i] = 0
        testdata = ina
        return testdata

In [ ]:
def createBarcharts(data):
        female_destinations = data.loc[data['gender'] == 'FEMALE' , 'country_destination' ].value_counts() ##value counts is a datafrmae, without it is a series
        female_destinations.plot('bar', color='r',  position = 0,label = 'feMale' )
        male_destination = data.loc[data['gender'] == 'MALE' , 'country_destination' ].value_counts() ##value counts is a datafrmae, without it is a series
        male_destination.plot('bar', color = 'b' , position = 1 , label = 'Male')
        #male_destination = data.loc[data['gender'] == 'other' , 'country_destination' ].value_counts() ##value counts is a datafrmae, without it is a series
        unknown_destinations = data.loc[data['gender'] == '-unknown-' , 'country_destination' ].value_counts() ##value counts is a datafrmae, without it is a series
        #male_destination.plot('bar', color = 'b' , position = 1 , label = 'Male')
        unknown_destinations.plot('bar', color='g',  position = 0.5,label = 'feMale' )

In [ ]:
def allvcounts(data):
    aggregate = pd.DataFrame(np.empty((0,0)))
    for attribute in data.axes[1]:
        tolist = data.loc[ :  , attribute ] .value_counts()
        aggregate = pd.concat((aggregate , tolist) , axis = 1)
    return aggregae

In [ ]:
def otherunk(data):
    other_destination = data.loc[data['gender'] == 'other' , 'country_destination' ].value_counts() ##value counts is a datafrmae, without it is a series
    unknown_destinations = data.loc[data['gender'] == '-unknown-' , 'country_destination' ].value_counts() ##value counts is a datafrmae, without it is a series
    a = other_destination.plot('bar', color = 'b' , position = 1 , label = 'Male')
    b = unknown_destinations.plot('bar', color='r',  position = 0.25,label = 'feMale' )
    return a ,b

In [ ]:
##create a one hot encodeing of all categorical variables 

In [ ]:
import matplotlib.dates as dates
data['date_first_booking']
dates.datestr2num(data['date_first_booking'])

In [ ]:
#ata['age'].fillna(-1)
#data['date_first_booking'].fillna(-1)




In [ ]:
#split into train and test set
ohe = pd.concat((ohe, data[['year_created' , 'month_created' , 'date_created' , 'year_booked', 'month_booked' , 'day_booked']]))
allb = ohe.loc[data['year_booked'] != -1 , :]
allb = allb.fillna(0)
valid = allb.iloc[0:10000 , :]
train = allb.iloc[10000:: , :]
dt = pd.concat((data['country_destination'] ,allb , ))











In [ ]:
mindata.axes

In [ ]:
#ohe , testdata are my two datasets not spilit into valid set
svc = SVC()
#sgd = SGDClassifier()
svc.fit(train , train_y).score(valid, valid_y)

In [ ]:
allb = allb.fillna(0)


In [ ]:
data.age.dropna().value_counts().plot(kind = 'hist' ,bins = 100)

In [ ]:
pd.DataFrame(data.age.dropna()).plot(kind ='bar')

In [ ]:
#create age percentiles
oldest = data.loc[data.age>42  , 'country_destination']
#middleupper = pd.concat((data.age<=42,data.age>33) , join = 'inner')
#data.age>33
#three = data.age<=33
#tweight = data.age>28
#middleyoung = pd.concat ( (pd.concat((three, tweight) , join = 'inner') , data['country_destination']) , join = 'inner')
youngest = data.loc[data.age<=28 , 'country_destination']
#midupper = data.loc[33<data.age<=42, 'country_destination']
old2 = data.loc[data.age<=33 , 'country_destination']
young2 = data.loc[data.age>33 , 'country_destination']

oldest.value_counts().plot('bar' ,color = 'b', position = 0)
youngest.value_counts().plot('bar', color = 'r', position = 1)

In [ ]:
# oldest and youngest vs country pref

#middleupper.value_counts().plot('bar', color = 'g' , position = 2)
#middleyoung.value_counts().plot('bar', color = 'y' , position = 3)

In [ ]:
##young and old raw vs country pref
old2.value_counts().plot('bar' , color = 'b' , position = 0)
young2.value_counts().plot('bar' , color = 'r' , position = 1)
total = len(data['gender'])
totalold = len(old2)
totalyoung = len(young2)
centold = old2.value_counts()/totalold
centyoung = young2.value_counts()/totalyoung


In [ ]:
## get years and months as a number so it can be a category 
def getyear(x):
    if type(x) == float:
        return -1
    else :
        return int(x[0:4])
def getmonth(x):
    if type(x) == float:
        return -1
    else :
        return int(x[5:7])
def getday(x):
    if type(x) == float:
        return -1
    else :
        return int(x[8:11])
def getorder(x):
    return( (x[0:4],x[5:7],x[8:11]) )
def getdate(x):
    return datetime.date(x).toordinal()
##add the column to data 
data['date_account_created2'] = data['date_account_created'].apply(getorder)
data['year_created'] = data['date_account_created'].apply(getyear)
data['month_created'] = data['date_account_created'].apply(getmonth)
data['date_created'] = data['date_account_created'].apply(getday)
data['year_booked'] = data['date_first_booking'].apply(getyear)
data['month_booked'] = data['date_first_booking'].apply(getmonth)
data['day_booked'] = data['date_first_booking'].apply(getday)
#ohe = pd.concat((ohe, data[['year_created' , 'month_created' , 'date_created' , 'year_booked', 'month_booked' , 'day_booked']]))

In [ ]:
centold.sum()

In [ ]:
allbooked = ohe.loc[data['year_booked'] != -1 , :]

In [ ]:
allbooked.axes

In [ ]:
train.isnull()

In [ ]:
ohe.info()

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC

In [ ]:
ohe = pd.concat((ohe, data[['year_created' , 'month_created' , 'date_created' , 'year_booked', 'month_booked' , 'day_booked']]))

In [ ]:
dates= data[['year_created' , 'month_created' , 'date_created' , 'year_booked', 'month_booked' , 'day_booked']]

In [ ]:
dates.info()

In [ ]:
##every month i want  to create a bin 
dates[['month_created' , 'year_created' ]].plot(kind = 'bar')
##i want number of signups to be on the x-axis , 


In [ ]:
def timeplots(datenums,numbooked):
    datenums.plot(kind = 'hist')
    numbooked.plot(kind = 'hist' , bins = 150)

In [ ]:
( data['age'] < 100 )

In [ ]:
import matplotlib.pyplot as plt
#plt.hist(datenums, bins = 20) 
try:
    plt.hist(numbooked['date_first_booking'])
except: KeyError

In [ ]:
total = len(data['language'] .dropna())
n = 0
def languageplot(data):
    percentsnon = data.loc[data['language'] != 'en' , 'country_destination'].value_counts()/total
    percentsen = data.loc[data['language'] == 'en' , 'country_destination'].value_counts()/total

#percents = data.loc[ : , 'country_destination'].value_counts()/total
    percentsnon.plot(kind = 'bar' , position = 0, color = 'b')
    percentsen.plot(kind = 'bar' , position = 0.5 , color = 'r')
    

In [ ]:
pd.get_dummies(data[['gender' , 'signup_method', 'signup_flow' , 'language' , 'affiliate_channel' , 'affiliate_provider' , 'first_affiliate_tracked', 'signup_app', 'first_device_type' , 'first_browser' ]]).axes

In [ ]:
data.loc['language' =='en'  , 'country_destination']

In [ ]:
#old2 = data.loc[data.gender , 'country_destination']
%pylab inline

In [ ]:
data['language']

In [ ]:
from sklearn.svm import SVC
svc = SVC()
#sgd = SGDClassifier()
svc.fit(train , trainy).score(valid, validy)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=600,max_depth = 6, criterion='gini')
rf.fit(train , trainy).score(valid, validy)

In [ ]:
len(y)

In [ ]:
len(trainy)

In [ ]:
len(data.iloc[0:10000])


In [ ]:
mindata = pd.get_dummies(nndata[['gender']])


In [ ]:
train

In [ ]:
trainy